## PRIMER PASO: BARAJEAR DATOS

## SETUP - Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
!pip install deep-translator pandas tqdm seaborn nltk scikit-learn gensim matplotlib numpy transformers torch

print("✓ Todas las dependencias instaladas correctamente")

In [ ]:
# Crear carpetas para organizar los archivos del proyecto
import os

folders = ['data', 'data_processed', 'models', 'charts']

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    
print("✓ Estructura de carpetas creada:")
print("  📁 data/              - Datasets originales")
print("  📁 data_processed/    - Datos procesados (CSV)")
print("  📁 models/            - Modelos entrenados")
print("  📁 charts/            - Gráficos generados")

## Mezclado

In [ ]:
import pandas as pd
import random

# Cargar dataset original
df = pd.read_csv('data/initial_data.csv')

print(f"Dataset original cargado: {len(df)} noticias")
print(f"Distribución de sentimientos:")
print(df['Sentiment'].value_counts())
print("\n" + "="*60)

# Crear copia para noticias correctas (50%)
df_correctas = df.copy()
df_correctas['Etiqueta'] = 'correcta'

# Separar noticias por sentimiento
df_positive = df[df['Sentiment'] == 'positive']
df_negative = df[df['Sentiment'] == 'negative']

print(f"\nNoticias positivas disponibles: {len(df_positive)}")
print(f"Noticias negativas disponibles: {len(df_negative)}")

# Extraer fragmentos de una noticia dividiendo por comas o puntos
def extraer_fragmentos(noticia):
    fragmentos = [p.strip() for p in noticia.split(',') if p.strip()]
    
    if len(fragmentos) <= 1:
        fragmentos = [p.strip() for p in noticia.split('.') if p.strip()]
    
    return fragmentos

# Crear noticias incorrectas mezclando fragmentos de sentimientos opuestos
def crear_mezclas_opuestas(df_positive, df_negative, num_mezclas):
    noticias_mezcladas = []
    
    print(f"\nCreando {num_mezclas} mezclas de sentimientos opuestos...")
    
    for i in range(num_mezclas):
        if (i + 1) % 1000 == 0:
            print(f"  Procesadas {i + 1}/{num_mezclas}...")
        
        # Seleccionar una noticia positiva y una negativa aleatoriamente
        idx_pos = random.randint(0, len(df_positive) - 1)
        idx_neg = random.randint(0, len(df_negative) - 1)
        
        noticia_pos = df_positive.iloc[idx_pos]['Sentence']
        noticia_neg = df_negative.iloc[idx_neg]['Sentence']
        
        # Extraer fragmentos de cada noticia
        frag_pos = extraer_fragmentos(noticia_pos)
        frag_neg = extraer_fragmentos(noticia_neg)
        
        # Seleccionar 1-2 fragmentos de cada noticia
        if len(frag_pos) > 0 and len(frag_neg) > 0:
            num_pos = min(random.randint(1, 2), len(frag_pos))
            num_neg = min(random.randint(1, 2), len(frag_neg))
            
            fragmentos_seleccionados = (
                random.sample(frag_pos, num_pos) + 
                random.sample(frag_neg, num_neg)
            )
            
            # Mezclar el orden de los fragmentos
            random.shuffle(fragmentos_seleccionados)
            
            noticia_mezclada = ', '.join(fragmentos_seleccionados)
            if noticia_mezclada and noticia_mezclada[-1] not in '.!?':
                noticia_mezclada += '.'
            
            # Asignar sentimiento aleatorio
            sentiment = random.choice(['positive', 'negative'])
            
            noticias_mezcladas.append({
                'Sentence': noticia_mezclada,
                'Sentiment': sentiment,
                'Etiqueta': 'incorrecta'
            })
    
    return pd.DataFrame(noticias_mezcladas)

# Crear el mismo número de noticias incorrectas que correctas
num_mezclas = len(df_correctas)
df_incorrectas = crear_mezclas_opuestas(df_positive, df_negative, num_mezclas)

# Combinar datasets
df_final = pd.concat([df_correctas, df_incorrectas], ignore_index=True)

# Mezclar aleatoriamente
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar estadísticas
print(f"\n{'='*60}")
print(f"ESTADÍSTICAS DEL DATASET FINAL")
print(f"{'='*60}")
print(f"Total de filas: {len(df_final)}")
print(f"Noticias correctas: {len(df_final[df_final['Etiqueta'] == 'correcta'])} ({len(df_final[df_final['Etiqueta'] == 'correcta'])/len(df_final)*100:.1f}%)")
print(f"Noticias incorrectas: {len(df_final[df_final['Etiqueta'] == 'incorrecta'])} ({len(df_final[df_final['Etiqueta'] == 'incorrecta'])/len(df_final)*100:.1f}%)")

print(f"\nDistribución de sentimientos en noticias CORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'correcta']['Sentiment'].value_counts())

print(f"\nDistribución de sentimientos en noticias INCORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'incorrecta']['Sentiment'].value_counts())

# Mostrar ejemplos
print(f"\n{'='*60}")
print(f"EJEMPLOS")
print(f"{'='*60}")

print("\n[NOTICIA CORRECTA]")
ejemplo_correcta = df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentiment']}")
print(ejemplo_correcta[:300] + ("..." if len(ejemplo_correcta) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 1]")
ejemplo_inc1 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentiment']}")
print(ejemplo_inc1[:300] + ("..." if len(ejemplo_inc1) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 2]")
ejemplo_inc2 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentiment']}")
print(ejemplo_inc2[:300] + ("..." if len(ejemplo_inc2) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 3]")
ejemplo_inc3 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentiment']}")
print(ejemplo_inc3[:300] + ("..." if len(ejemplo_inc3) > 300 else ""))

# Guardar dataset final
output_path = 'data/dataset_mezclado_final.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n{'='*60}")
print(f"✓ Dataset guardado como '{output_path}'")
print(f"{'='*60}")

## TRADUCCIÓN

In [ ]:
# Traducir el dataset a múltiples idiomas usando Google Translator
import pandas as pd
from deep_translator import GoogleTranslator
import random
from tqdm import tqdm
import time
import os

# Cargar dataset
df = pd.read_csv('data/dataset_mezclado_final.csv')

# Idiomas disponibles
idiomas = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'eu', 'gl']

nombres_idiomas = {
    'es': 'español',
    'en': 'inglés',
    'fr': 'francés',
    'de': 'alemán',
    'it': 'italiano',
    'pt': 'portugués',
    'ca': 'catalán',
    'eu': 'euskera',
    'gl': 'gallego'
}

df_traducido = df.copy()
df_traducido['Idioma'] = ''

# Traducir texto con manejo de errores
def traducir_texto(texto, idioma_destino):
    try:
        if idioma_destino == 'es':
            return texto
        translator = GoogleTranslator(source='es', target=idioma_destino)
        traduccion = translator.translate(texto)
        time.sleep(0.5)
        return traduccion
    except Exception as e:
        print(f"Error traduciendo a {idioma_destino}: {e}")
        return texto

# Traducir solo la columna 'Sentence' a un idioma aleatorio por fila
print("Iniciando traducción del dataset...")
print(f"Total de filas a procesar: {len(df_traducido)}")

for idx in tqdm(range(len(df_traducido))):
    # Seleccionar idioma aleatorio (mayor probabilidad para español)
    idioma_elegido = random.choice(idiomas + ['es', 'es', 'es'])
    
    # Traducir la columna 'Sentence'
    texto_original = df_traducido.loc[idx, 'Sentence']
    df_traducido.loc[idx, 'Sentence'] = traducir_texto(texto_original, idioma_elegido)
    df_traducido.loc[idx, 'Idioma'] = nombres_idiomas[idioma_elegido]

# Guardar dataset traducido
output_path = 'data/dataset_multiidioma.csv'
df_traducido.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✓ Dataset traducido guardado en: {output_path}")
print(f"Total de filas: {len(df_traducido)}")
print(f"\nEstructura del dataset:")
print(df_traducido.head())
print(f"\nDistribución de idiomas:")
print(df_traducido['Idioma'].value_counts())

## 1. Análisis Exploratorio de Datos (EDA)

In [ ]:
import pandas as pd
import numpy as np

print("Iniciando Análisis Exploratorio de Datos...")
try:
    df = pd.read_csv('data/dataset_multiidioma.csv', encoding='utf-8')
except FileNotFoundError:
    print("\nERROR: No se encontró 'dataset_multiidioma.csv'.")
    exit()

print("=" * 60)
print(" RESUMEN DEL DATASET ")
print("=" * 60)

# Mostrar información básica del dataset
print("\n1. ESTRUCTURA Y CONTEO DE VALORES NO NULOS:")
df.info()

# Estadísticas descriptivas
print("\n2. ESTADÍSTICAS DESCRIPTIVAS:")
print("-" * 60)
print(df.describe(include='all'))

# Distribuciones de variables categóricas
print("\n3. DISTRIBUCIONES CLAVE (Conteo y Porcentaje):")
for col in ['Sentiment', 'Etiqueta', 'Idioma']:
    counts = df[col].value_counts().rename('Conteo')
    percents = df[col].value_counts(normalize=True).mul(100).round(2).rename('Porcentaje (%)')
    print(f"\n--- Distribución de: {col} ---")
    print(pd.concat([counts, percents], axis=1))

# Calcular longitud de oraciones en palabras
df['sentence_length'] = df['Sentence'].str.split().str.len()
print("\n4. ESTADÍSTICAS DE LONGITUD DE ORACIONES (En palabras):")
print("-" * 60)
print(df['sentence_length'].describe().round(2))

# Análisis de valores nulos
print("\n5. ANÁLISIS DE VALORES NULOS:")
print("-" * 60)
nulos = df.isnull().sum()
if nulos.sum() == 0:
    print("✓ No hay valores nulos en el dataset")
else:
    print(nulos[nulos > 0])

# Análisis de duplicados
print("\n6. ANÁLISIS DE DUPLICADOS:")
print("-" * 60)
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados} ({duplicados/len(df)*100:.2f}%)")
if duplicados > 0:
    print(f"Filas únicas: {len(df) - duplicados}")

# Análisis cruzado entre variables
print("\n7. ANÁLISIS CRUZADO: Sentiment vs Etiqueta")
print("-" * 60)
crosstab = pd.crosstab(df['Sentiment'], df['Etiqueta'], margins=True)
print("\nConteo absoluto:")
print(crosstab)
print("\nPorcentaje por fila:")
crosstab_pct = pd.crosstab(df['Sentiment'], df['Etiqueta'], normalize='index') * 100
print(crosstab_pct.round(2))

## 2. Visualizaciones del Dataset

In [ ]:
# Crear visualizaciones del dataset
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set_theme(style="whitegrid", palette="husl")

print("=" * 60)
print("VISUALIZACIONES DEL DATASET")
print("=" * 60)

# 1. Distribuciones principales
print("\n1. DISTRIBUCIONES PRINCIPALES")
print("-" * 60)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribuciones del Dataset', fontsize=16, fontweight='bold')

# Gráfico de sentimientos
sns.countplot(x='Sentiment', data=df, ax=axes[0], order=df['Sentiment'].value_counts().index)
axes[0].set_title('Distribución de Sentimientos')
axes[0].set_xlabel('Sentimiento')
axes[0].set_ylabel('Frecuencia')

# Gráfico de idiomas (top 5)
top_idiomas = df['Idioma'].value_counts().head(5).index
df_top_idiomas = df[df['Idioma'].isin(top_idiomas)]
sns.countplot(data=df_top_idiomas, y='Idioma', order=top_idiomas, ax=axes[1], palette='viridis')
axes[1].set_title('Top 5 Idiomas')
axes[1].set_xlabel('Frecuencia')

# Gráfico de etiquetas
colors = ['#2ecc71', '#e74c3c']
sns.countplot(data=df, x='Etiqueta', ax=axes[2], palette=colors)
axes[2].set_title('Distribución de Etiquetas')
axes[2].set_xlabel('Etiqueta')
axes[2].set_ylabel('Frecuencia')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/01_distribuciones.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 01_distribuciones.png")
plt.show()

# 2. Análisis cruzado
print("\n2. ANÁLISIS CRUZADO")
print("-" * 60)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
fig.suptitle('Relación Sentimiento vs Etiqueta', fontsize=16, fontweight='bold')

# Crear heatmap de porcentajes
pivot_pct = pd.crosstab(df['Sentiment'], df['Etiqueta'], normalize='index') * 100
sns.heatmap(pivot_pct, annot=True, fmt='.1f', cmap='RdYlGn_r', ax=ax, 
            cbar_kws={'label': 'Porcentaje (%)'})
ax.set_title('Porcentaje por Sentimiento')
ax.set_xlabel('Etiqueta')
ax.set_ylabel('Sentimiento')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/02_heatmap_sentiment_etiqueta.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 02_heatmap_sentiment_etiqueta.png")
plt.show()

# 3. Análisis de longitud de oraciones
print("\n3. ANÁLISIS DE LONGITUD")
print("-" * 60)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Análisis de Longitud de Oraciones', fontsize=16, fontweight='bold')

# Histograma de longitudes
sns.histplot(df['sentence_length'], bins=30, kde=True, ax=axes[0])
media = df['sentence_length'].mean()
axes[0].axvline(media, color='red', linestyle='--', label=f'Media: {media:.1f}')
axes[0].set_title('Distribución de Longitud')
axes[0].set_xlabel('Número de Palabras')
axes[0].legend()

# Boxplot por sentimiento
sns.boxplot(x='Sentiment', y='sentence_length', data=df, ax=axes[1])
axes[1].set_title('Longitud por Sentimiento')
axes[1].set_xlabel('Sentimiento')
axes[1].set_ylabel('Número de Palabras')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/03_analisis_longitud.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 03_analisis_longitud.png")
plt.show()

# Resumen
print("\n" + "=" * 60)
print("RESUMEN DE VISUALIZACIONES")
print("=" * 60)
print("✓ 01_distribuciones.png")
print("✓ 02_heatmap_sentiment_etiqueta.png")
print("✓ 03_analisis_longitud.png")

## 3. Preprocesamiento de Datos - Tokenización y Limpieza

In [ ]:
import re
import pandas as pd
import string
import nltk

# Descargar recursos de NLTK
try:
    print("Verificando recursos NLTK esenciales...")
    nltk.download('punkt', quiet=True, raise_on_error=False)
    nltk.download('stopwords', quiet=True, raise_on_error=False)
    print("✓ Recursos NLTK (punkt, stopwords) listos.")
except Exception:
    print("ATENCIÓN: La descarga de recursos de NLTK falló. Usaremos tokenización simple.")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

# Tokenización simple como alternativa
def simple_tokenize(text):
    return re.findall(r"[\w']+|[.,!?;]", text.lower())

print("\n" + "=" * 80)
print("PREPROCESAMIENTO DE DATOS")
print("=" * 80)

data = 'data/dataset_multiidioma.csv'

# Cargar stopwords en múltiples idiomas
try:
    STOPWORDS_ES = set(stopwords.words('spanish'))
    STOPWORDS_EN = set(stopwords.words('english'))
    STOPWORDS_ALL = STOPWORDS_ES.union(STOPWORDS_EN) 
except LookupError:
    STOPWORDS_ALL = set()

df_processed = pd.read_csv(data).copy()

# Preprocesar texto: limpieza, tokenización y eliminación de stopwords
def preprocess_text(text, stopwords_set, min_len=2):
    if pd.isna(text) or not str(text).strip():
        return []
    
    text = str(text)
    
    # Eliminar URLs y menciones
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|#\w+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenizar y convertir a minúsculas
    try:
        tokens = word_tokenize(text.lower()) 
    except LookupError:
        tokens = simple_tokenize(text)

    # Eliminar puntuación, stopwords y tokens cortos
    tokens_final = [
        token for token in tokens
        if token not in string.punctuation 
        and token not in stopwords_set
        and len(token) > min_len
    ]
    
    return tokens_final

# Aplicar preprocesamiento
print("\n1. APLICANDO PROCESO DE LIMPIEZA...")
print("-" * 80)

df_processed['tokens_processed'] = df_processed['Sentence'].apply(
    lambda x: preprocess_text(x, STOPWORDS_ALL)
)

df_processed['tokens_original'] = df_processed['Sentence'].apply(
     lambda x: simple_tokenize(str(x)) if not pd.isna(x) else []
)

print("✓ Preprocesamiento completado en la columna 'tokens_processed'")

# Mostrar resultados
print("\n" + "=" * 80)
print("RESULTADOS DEL PREPROCESAMIENTO")
print("=" * 80)

print("\n✅ Columna de tokens procesados (Primeras 5 filas):")
print(df_processed[['Sentence', 'tokens_processed']].head())

print("-" * 80)

# Ejemplo de transformación
if len(df_processed) > 0:
    print("\nEjemplo de transformación de la primera fila:")
    print(f"  Texto Original: {df['Sentence'].iloc[0]}")
    print(f"  Tokens Finales: {df_processed['tokens_processed'].iloc[0]}")

print("-" * 80)

# Calcular estadísticas de reducción
df_processed['num_tokens_original'] = df_processed['tokens_original'].apply(len)
df_processed['num_tokens_processed'] = df_processed['tokens_processed'].apply(len)

mean_original = df_processed['num_tokens_original'].mean()
mean_processed = df_processed['num_tokens_processed'].mean()
reduction_percentage = ((mean_original - mean_processed) / mean_original * 100) if mean_original > 0 else 0

print(f"\nEstadísticas de Longitud:")
print(f"  Promedio de tokens originales: {mean_original:.2f}")
print(f"  Promedio de tokens procesados: {mean_processed:.2f}")
print(f"  Reducción de ruido promedio: **{reduction_percentage:.2f}%**")

# Guardar resultado
df_processed.to_csv('data_processed/datos_preprocesados_simple.csv', index=False)
print(f"\n✓ Datos guardados en 'data_processed/datos_preprocesados_simple.csv'")

## 4. Lemmatization y Stemming

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import nltk
import numpy as np
import ast

# Descargar recursos de NLTK
try:
    print("Verificando recursos NLTK esenciales (wordnet)...")
    nltk.download('wordnet', quiet=True, raise_on_error=False)
    print("✓ Recurso 'wordnet' listo.")
except Exception:
    print("ATENCIÓN: El recurso 'wordnet' de NLTK no está disponible.")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

print("\n" + "=" * 60)
print("LEMMATIZATION Y STEMMING")
print("=" * 60)

# Cargar datos
df_processed = pd.read_csv('data_processed/datos_preprocesados_simple.csv').copy()

# Convertir strings de listas a listas reales
print("\n1. CONVIRTIENDO TOKENS (STRING → LISTA)")
print("-" * 60)

def convert_to_list(value):
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            result = ast.literal_eval(value)
            if isinstance(result, list):
                return result
        except:
            return value.split()
    return []

# Aplicar conversión a las columnas de tokens
df_processed['tokens_processed'] = df_processed['tokens_processed'].apply(convert_to_list)
df_processed['tokens_original'] = df_processed['tokens_original'].apply(convert_to_list)

# Verificar conversión
sample = df_processed['tokens_processed'].iloc[0]
if isinstance(sample, list) and len(sample) > 0 and isinstance(sample[0], str):
    vocab_real = len(set(token for tokens in df_processed['tokens_processed'] for token in tokens))
    print(f"✓ Conversión exitosa")
    print(f"  Tipo: {type(sample)}")
    print(f"  Ejemplo: {sample[:5]}")
    print(f"  Vocabulario real: {vocab_real} palabras")
else:
    print("⚠️  ADVERTENCIA: La conversión puede tener problemas")

# Inicializar herramientas de stemming y lemmatization
porter_stemmer = PorterStemmer()
word_lemmatizer = WordNetLemmatizer()

# Aplicar stemming
print("\n2. STEMMING (Porter Stemmer)")
print("-" * 60)

df_processed['tokens_stemmed'] = df_processed['tokens_processed'].apply(
    lambda tokens: [porter_stemmer.stem(token) for token in tokens] if isinstance(tokens, list) else []
)
print("✓ Stemming completado")

# Aplicar lemmatization
print("\n3. LEMMATIZATION (WordNet Lemmatizer)")
print("-" * 60)

df_processed['tokens_lemmatized'] = df_processed['tokens_processed'].apply(
    lambda tokens: [word_lemmatizer.lemmatize(token, pos='v') for token in tokens] if isinstance(tokens, list) else []
)
print("✓ Lemmatization completada")

# Comparar resultados
print("\n4. COMPARACIÓN DE RESULTADOS")
print("-" * 60)

if not df_processed.empty and len(df_processed['tokens_processed'].iloc[0]) > 0:
    original = df_processed['tokens_processed'].iloc[0][:3]
    stemmed = df_processed['tokens_stemmed'].iloc[0][:3]
    lemmatized = df_processed['tokens_lemmatized'].iloc[0][:3]
    
    print(f"{'Token Original':<20} {'Stemming':<20} {'Lemmatization':<20}")
    print("-" * 60)
    for i in range(min(len(original), 3)):
        print(f"{original[i]:<20} {stemmed[i]:<20} {lemmatized[i]:<20}")

# Calcular tamaño de vocabulario después de cada técnica
vocab_original = set(token for tokens in df_processed['tokens_processed'] for token in tokens if isinstance(tokens, list))
vocab_stemmed = set(token for tokens in df_processed['tokens_stemmed'] for token in tokens if isinstance(tokens, list))
vocab_lemmatized = set(token for tokens in df_processed['tokens_lemmatized'] for token in tokens if isinstance(tokens, list))

print(f"\nTamaño del vocabulario base: {len(vocab_original)}")
print(f"Tamaño después de Stemming: {len(vocab_stemmed)} ({((len(vocab_original) - len(vocab_stemmed)) / len(vocab_original) * 100):.2f}% reducción)")
print(f"Tamaño después de Lemmatization: {len(vocab_lemmatized)} ({((len(vocab_original) - len(vocab_lemmatized)) / len(vocab_original) * 100):.2f}% reducción)")

# Convertir tokens a texto para compatibilidad
df_processed['text_stemmed'] = df_processed['tokens_stemmed'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)
df_processed['text_lemmatized'] = df_processed['tokens_lemmatized'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)
df_processed['text_processed_base'] = df_processed['tokens_processed'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)

# Guardar
df_processed.to_csv('data_processed/datos_preprocesados_completo.csv', index=False)
print(f"\n✓ Datos guardados en 'data_processed/datos_preprocesados_completo.csv'")

## 5. Representación Tradicional: Bag of Words (BoW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="viridis")

print("\n" + "=" * 60)
print("VECTORIZACIÓN: BAG OF WORDS (BoW)")
print("=" * 60)

# Seleccionar columna de texto procesado
TEXT_COLUMN = 'text_lemmatized'

# Crear vectorizador con parámetros para reducir ruido
bow_vectorizer = CountVectorizer(
    max_features=5000, 
    min_df=2, 
    max_df=0.8, 
    ngram_range=(1, 2)
)

# Crear matriz BoW
X_bow = bow_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
feature_names = bow_vectorizer.get_feature_names_out()

print(f"✓ Matriz BoW creada. Forma: {X_bow.shape}")
print(f"Tamaño del Vocabulario Final: {X_bow.shape[1]}")

# Calcular frecuencias de términos
print("\n2. ANÁLISIS DE FRECUENCIAS (TOP 15)")
print("-" * 60)

word_freq = np.asarray(X_bow.sum(axis=0)).flatten()
freq_df = pd.DataFrame({
    'word': feature_names,
    'frequency': word_freq
}).sort_values('frequency', ascending=False).head(15)

print(freq_df)

# Visualizar términos más frecuentes
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='word', data=freq_df)
plt.title(f'Top 15 Términos Más Frecuentes (BoW)', fontweight='bold')
plt.tight_layout()
plt.savefig('charts/bow_top_terms.png', dpi=300)
plt.show()

# Analizar términos clave por sentimiento
print("\n3. TÉRMINOS CLAVE POR SENTIMIENTO (Top 5 por Clase)")
print("-" * 60)

for sentiment in df_processed['Sentiment'].unique():
    # Filtrar documentos por sentimiento
    docs_sentiment = df_processed[df_processed['Sentiment'] == sentiment][TEXT_COLUMN]
    
    # Vectorizar documentos
    X_sentiment = bow_vectorizer.transform(docs_sentiment)
    
    # Calcular frecuencias
    freq_sentiment = np.asarray(X_sentiment.sum(axis=0)).flatten()
    
    # Mostrar top 5 términos
    freq_df_sentiment = pd.DataFrame({
        'word': feature_names,
        'frequency': freq_sentiment
    }).sort_values('frequency', ascending=False).head(5)
    
    print(f"\n{sentiment.upper()}:")
    print(freq_df_sentiment)

# Preparar datos finales para el modelo
print("\n4. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Convertir matriz dispersa a DataFrame
X_dense = X_bow.toarray()
bow_final_df = pd.DataFrame(X_dense, columns=feature_names)

# Añadir columnas de etiquetas
bow_final_df['Sentiment'] = df_processed['Sentiment'].values
bow_final_df['Idioma'] = df_processed['Idioma'].values

# Guardar
bow_final_df.to_csv('data_processed/datos_vectorizados_final.csv', index=False)
print(f"✓ Matriz de características guardada en 'data_processed/datos_vectorizados_final.csv'")

## 6. Representación Tradicional: TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

print("\n" + "=" * 60)
print("VECTORIZACIÓN: TF-IDF")
print("=" * 60)

# Seleccionar columna de texto procesado
TEXT_COLUMN = 'text_lemmatized'

# Crear vectorizador TF-IDF
print("\n1. CREANDO MATRIZ TF-IDF")
print("-" * 60)

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True
)

# Crear matriz TF-IDF
try:
    X_tfidf = tfidf_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
except NameError:
    print("ERROR: El DataFrame 'df_processed' no está definido. Asegúrate de cargar los datos antes.")
    sys.exit()

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"✓ Modelo TF-IDF creado. Forma: {X_tfidf.shape}")
print(f"Tamaño del Vocabulario Final: {X_tfidf.shape[1]}")

# Analizar términos con mayor peso TF-IDF
print("\n2. TÉRMINOS CON MAYOR PESO TF-IDF PROMEDIO (TOP 15)")
print("-" * 60)

# Calcular peso promedio de cada término
tfidf_means = np.asarray(X_tfidf.mean(axis=0)).flatten()
tfidf_df = pd.DataFrame({
    'term': tfidf_feature_names,
    'tfidf_mean': tfidf_means
}).sort_values('tfidf_mean', ascending=False).head(15)

print(tfidf_df)

# Preparar datos finales
print("\n3. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Convertir matriz dispersa a DataFrame
tfidf_matrix_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_feature_names)
tfidf_matrix_df['Sentiment'] = df_processed['Sentiment'].values

# Guardar
tfidf_matrix_df.to_csv('data_processed/datos_tfidf_final.csv', index=False)
print(f"✓ Matriz de características guardada en 'data_processed/datos_tfidf_final.csv'")

## 7. Word Embeddings

In [ ]:
# Crear embeddings usando Word2Vec y FastText
from gensim.models import Word2Vec, FastText
import pandas as pd
import numpy as np

print("=" * 60)
print("WORD EMBEDDINGS NO CONTEXTUALES")
print("=" * 60)

# Cargar datos procesados
df_processed = pd.read_csv('data_processed/datos_preprocesados_completo.csv')

# Convertir tokens a listas
import ast

def ensure_list(value):
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except:
            return value.split()
    return []

df_processed['tokens_lemmatized'] = df_processed['tokens_lemmatized'].apply(ensure_list)
sentences = df_processed['tokens_lemmatized'].tolist()

sample = sentences[0]
print(f"\nVerificación inicial:")
print(f"  Tipo: {type(sample)}")
print(f"  Ejemplo: {sample[:5] if len(sample) > 0 else 'vacío'}")

# WORD2VEC
print("\n" + "=" * 60)
print("1. WORD2VEC")
print("=" * 60)

print("\nEntrenando modelo Word2Vec...")
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

vocab_size = len(w2v_model.wv)
print(f"✓ Modelo entrenado")
print(f"  Vocabulario: {vocab_size} palabras")
print(f"  Dimensión: {w2v_model.wv.vector_size}")

# Calcular palabras fuera de vocabulario (OOV)
oov_count = sum(1 for tokens in sentences for token in tokens if token not in w2v_model.wv)
total_tokens = sum(len(tokens) for tokens in sentences)
oov_percentage = (oov_count / total_tokens * 100) if total_tokens > 0 else 0

print(f"\nAnálisis OOV:")
print(f"  Palabras fuera de vocabulario: {oov_count:,}/{total_tokens:,}")
print(f"  Porcentaje OOV: {oov_percentage:.2f}%")

# Generar vectores de documentos promediando los vectores de palabras
def get_document_vector_w2v(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

print("\nGenerando vectores de documentos...")
doc_vectors_w2v = df_processed['tokens_lemmatized'].apply(
    lambda tokens: get_document_vector_w2v(tokens, w2v_model)
)

X_w2v = np.vstack(doc_vectors_w2v.values)
w2v_df = pd.DataFrame(X_w2v, columns=[f'w2v_{i}' for i in range(X_w2v.shape[1])])
w2v_df['Sentiment'] = df_processed['Sentiment'].values
w2v_df['Idioma'] = df_processed['Idioma'].values

w2v_df.to_csv('data_processed/datos_word2vec.csv', index=False)
w2v_model.save('models/word2vec_model.model')
print("✓ Vectores Word2Vec guardados")

# Ejemplo de palabras similares
if vocab_size > 1000:
    try:
        test_words = ['company', 'profit', 'loss', 'market']
        print("\nEjemplos de palabras similares:")
        for word in test_words:
            if word in w2v_model.wv:
                similar = w2v_model.wv.most_similar(word, topn=3)
                print(f"  {word}: {[w for w, s in similar]}")
                break
    except:
        pass

# FASTTEXT
print("\n" + "=" * 60)
print("2. FASTTEXT")
print("=" * 60)

print("\nEntrenando modelo FastText...")
ft_model = FastText(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

vocab_size_ft = len(ft_model.wv)
print(f"✓ Modelo entrenado")
print(f"  Vocabulario: {vocab_size_ft} palabras")
print(f"  Dimensión: {ft_model.wv.vector_size}")

print(f"\nVentaja de FastText:")
print(f"  ✓ Todas las palabras tienen representación (incluso OOV)")
print(f"  ✓ Usa subwords para palabras desconocidas")
print(f"  Total de tokens procesados: {total_tokens:,}")

# Generar vectores de documentos (FastText no tiene problema con OOV)
def get_document_vector_ft(tokens, model):
    vectors = [model.wv[word] for word in tokens]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

print("\nGenerando vectores de documentos...")
doc_vectors_ft = df_processed['tokens_lemmatized'].apply(
    lambda tokens: get_document_vector_ft(tokens, ft_model)
)

X_ft = np.vstack(doc_vectors_ft.values)
ft_df = pd.DataFrame(X_ft, columns=[f'ft_{i}' for i in range(X_ft.shape[1])])
ft_df['Sentiment'] = df_processed['Sentiment'].values
ft_df['Idioma'] = df_processed['Idioma'].values

ft_df.to_csv('data_processed/datos_fasttext.csv', index=False)
ft_model.save('models/fasttext_model.model')
print("✓ Vectores FastText guardados")

# Ejemplo de palabras similares
if vocab_size_ft > 1000:
    try:
        test_words = ['company', 'profit', 'loss', 'market']
        print("\nEjemplos de palabras similares:")
        for word in test_words:
            similar = ft_model.wv.most_similar(word, topn=3)
            print(f"  {word}: {[w for w, s in similar]}")
            break
    except:
        pass

print("\n" + "=" * 60)
print("RESUMEN DE EMBEDDINGS NO CONTEXTUALES")
print("=" * 60)
print(f"✓ Word2Vec: {vocab_size:,} palabras, {oov_percentage:.2f}% OOV")
print(f"✓ FastText: {vocab_size_ft:,} palabras, 0% OOV (usa subwords)")

## 8. Word Embeddings Contextuales con BERT

In [ ]:
# Crear embeddings contextuales usando BERT
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

print("=" * 60)
print("WORD EMBEDDINGS CONTEXTUALES - BERT")
print("=" * 60)

# Cargar datos
df_processed = pd.read_csv('data_processed/datos_preprocesados_completo.csv')

# Cargar modelo BERT multilingüe
model_name = 'bert-base-multilingual-cased'
print(f"\nCargando modelo: {model_name}")
print("(Este proceso puede tardar unos minutos la primera vez)")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Usar GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

print(f"✓ Modelo cargado en: {device}")
print(f"  Vocabulario: {len(tokenizer)} tokens")
print(f"  Dimensión de embeddings: {model.config.hidden_size}")

# Obtener embedding de BERT usando el token [CLS]
def get_bert_embedding(text, tokenizer, model, device, max_length=128):
    # Tokenizar texto
    inputs = tokenizer(text, return_tensors='pt', truncation=True, 
                      padding=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Obtener embeddings sin calcular gradientes
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Usar el embedding del token [CLS] que representa toda la oración
    cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embedding.flatten()

# Generar embeddings para todos los documentos
print(f"\nGenerando embeddings para {len(df_processed)} documentos...")
print("(Esto puede tardar varios minutos)")

embeddings_list = []
batch_size = 32

for i in tqdm(range(0, len(df_processed), batch_size)):
    batch = df_processed['Sentence'].iloc[i:i+batch_size].tolist()
    
    for text in batch:
        embedding = get_bert_embedding(str(text), tokenizer, model, device)
        embeddings_list.append(embedding)

# Crear DataFrame con embeddings
X_bert = np.vstack(embeddings_list)
bert_df = pd.DataFrame(X_bert, columns=[f'bert_{i}' for i in range(X_bert.shape[1])])
bert_df['Sentiment'] = df_processed['Sentiment'].values
bert_df['Idioma'] = df_processed['Idioma'].values

# Guardar
bert_df.to_csv('data_processed/datos_bert.csv', index=False)
print(f"\n✓ Embeddings BERT guardados")
print(f"  Forma de la matriz: {X_bert.shape}")
print(f"  Archivo: datos_bert.csv")

print(f"\nAnálisis OOV:")
print(f"  ✓ BERT no tiene palabras OOV")
print(f"  ✓ Usa subword tokenization (WordPiece)")
print(f"  ✓ Embeddings contextuales (varía según contexto)")

print("\n" + "=" * 60)
print("COMPARACIÓN: NO CONTEXTUALES vs CONTEXTUALES")
print("=" * 60)
print("\nWord2Vec/FastText (No Contextuales):")
print("  • Cada palabra tiene UN SOLO vector")
print("  • No considera contexto")
print("  • Vocabulario limitado (problemas OOV)")
print("\nBERT (Contextual):")
print("  • Cada palabra tiene MÚLTIPLES vectores según contexto")
print("  • Considera contexto completo de la oración")
print("  • Sin problemas OOV (subword tokenization)")
print("  • Más costoso computacionalmente")